In [80]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

from tensorflow.keras.preprocessing.image import load_img
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from PIL import Image
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import image_dataset_from_directory

from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions

#  Images to train,val,test files

In [2]:
import os
class_names = sorted(os.listdir("./images"))

In [6]:
def paths():
    paths = list()
    for type in class_names:
        base_name = "./images/{}/".format(type)
        for img_name in os.listdir(base_name):
            paths.append(base_name + img_name)
    return paths

paths = paths()

In [7]:
train_paths, val_test_paths= train_test_split(paths, test_size=0.2, random_state=1029)
val_paths, test_paths = train_test_split(val_test_paths, test_size=0.5, random_state=1029)

In [10]:
def train_val_test_dataset(name, paths):
    os.mkdir('./' + name)
    for type in class_names:
        os.mkdir('./'+ name + '/' + type)
        
    for i in range(len(paths)):
        for type in class_names:
             if (paths[i].split('/')[2] == type):
                img = Image.open(paths[i])
                img.save('./'+ name + '/'+ type + '/' + paths[i].split('/')[3] )

In [11]:
train_val_test_dataset('train', train_paths)
train_val_test_dataset('val', val_paths)
train_val_test_dataset('test', test_paths)

# Train,val,test image processing with batch size 128 and input size 299

In [105]:
train_gen = ImageDataGenerator(rescale=1./255)

train_ds =  train_gen.flow_from_directory(
            './train',
            target_size=(150, 150),
            batch_size=64,
            class_mode='binary'
)

val_gen = ImageDataGenerator(rescale=1./255)

val_ds =  val_gen.flow_from_directory(
         './val',
          target_size=(150, 150),
          batch_size=64,
          shuffle=False,
          class_mode='binary'
)

test_gen = ImageDataGenerator(rescale=1./255)

test_ds = test_gen.flow_from_directory(
    './test',
    target_size=(150, 150),
    batch_size=64,
    shuffle=False,
    class_mode='binary'
)

Found 32000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


# Model

In [107]:
def make_model(learning_rate=0.01):
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',\
             optimizer=optimizers.SGD(learning_rate=learning_rate, momentum=0.8),\
             metrics=['acc'])

    return model
    

In [108]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_54 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 74, 74, 32)       0         
 g2D)                                                            
                                                                 
 flatten_6 (Flatten)         (None, 175232)            0         
                                                                 
 dense_30 (Dense)            (None, 64)                11214912  
                                                                 
 dense_31 (Dense)            (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
__________________________________________

# Model Tuning with learning rate

In [109]:
scores = {}

for lr in [ 0.0001, 0.001, 0.01, 0.1]:
    print(lr)

    model = make_model(learning_rate=lr)
    history = model.fit(train_ds, epochs=10, validation_data=val_ds)
    scores[lr] = history.history
    
    print()
    print()

0.0001
Epoch 1/10
500/500 [==============================] - 68s 134ms/step - loss: 0.6528 - acc: 0.6697 - val_loss: 0.6145 - val_acc: 0.8330
Epoch 2/10
500/500 [==============================] - 67s 134ms/step - loss: 0.5751 - acc: 0.7903 - val_loss: 0.5326 - val_acc: 0.8140
Epoch 3/10
500/500 [==============================] - 68s 135ms/step - loss: 0.5003 - acc: 0.8237 - val_loss: 0.4823 - val_acc: 0.8515
Epoch 4/10
500/500 [==============================] - 70s 139ms/step - loss: 0.4438 - acc: 0.8415 - val_loss: 0.4225 - val_acc: 0.8643
Epoch 5/10
500/500 [==============================] - 74s 147ms/step - loss: 0.4022 - acc: 0.8541 - val_loss: 0.3844 - val_acc: 0.8597
Epoch 6/10
500/500 [==============================] - 86s 171ms/step - loss: 0.3736 - acc: 0.8629 - val_loss: 0.3588 - val_acc: 0.8705
Epoch 7/10
500/500 [==============================] - 84s 168ms/step - loss: 0.3525 - acc: 0.8683 - val_loss: 0.3423 - val_acc: 0.8760
Epoch 8/10
500/500 [============================

KeyboardInterrupt: 

In [115]:
learning_rate = 0.01

# Adding droprate

In [116]:
def make_model(learning_rate=0.01):
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',\
             optimizer=optimizers.SGD(learning_rate=learning_rate, momentum=0.8),\
             metrics=['acc'])

    return model

In [117]:
scores = {}
model = make_model(learning_rate=learning_rate)
history = model.fit(train_ds, epochs=10, validation_data=val_ds)
scores[learning_rate] = history.history

Epoch 1/10
500/500 [==============================] - 73s 145ms/step - loss: 0.3931 - acc: 0.8398 - val_loss: 0.2284 - val_acc: 0.9287
Epoch 2/10
500/500 [==============================] - 71s 142ms/step - loss: 0.1415 - acc: 0.9652 - val_loss: 0.0795 - val_acc: 0.9768
Epoch 3/10
500/500 [==============================] - 77s 153ms/step - loss: 0.0868 - acc: 0.9788 - val_loss: 0.0489 - val_acc: 0.9895
Epoch 4/10
500/500 [==============================] - 70s 141ms/step - loss: 0.0614 - acc: 0.9835 - val_loss: 0.0472 - val_acc: 0.9847
Epoch 5/10
500/500 [==============================] - 60s 119ms/step - loss: 0.0498 - acc: 0.9865 - val_loss: 0.0333 - val_acc: 0.9898
Epoch 6/10
500/500 [==============================] - 59s 119ms/step - loss: 0.0452 - acc: 0.9867 - val_loss: 0.0321 - val_acc: 0.9905
Epoch 7/10
500/500 [==============================] - 69s 137ms/step - loss: 0.0398 - acc: 0.9892 - val_loss: 0.0306 - val_acc: 0.9902
Epoch 8/10
500/500 [==============================] - 6

# val_acc is good enough so I skipped image manipulation
# Next part is model saving

In [123]:
checkpoint = keras.callbacks.ModelCheckpoint(
    'final_model_{epoch:02d}_{val_acc:.3f}.h5',
    save_best_only=True,
    monitor='val_acc',
    mode='max'
)

# Training Final Model

In [124]:
learning_rate=0.01
model = make_model(learning_rate=learning_rate)
history = model.fit(train_ds, epochs=10, validation_data=val_ds, callbacks=[checkpoint])

Epoch 1/10
500/500 [==============================] - 76s 150ms/step - loss: 0.4666 - acc: 0.7618 - val_loss: 0.1736 - val_acc: 0.9440
Epoch 2/10
500/500 [==============================] - 71s 142ms/step - loss: 0.1464 - acc: 0.9524 - val_loss: 0.1877 - val_acc: 0.9815
Epoch 3/10
500/500 [==============================] - 61s 123ms/step - loss: 0.0975 - acc: 0.9747 - val_loss: 0.0581 - val_acc: 0.9820
Epoch 4/10
500/500 [==============================] - 71s 141ms/step - loss: 0.0831 - acc: 0.9797 - val_loss: 0.0563 - val_acc: 0.9885
Epoch 5/10
500/500 [==============================] - 67s 134ms/step - loss: 0.0591 - acc: 0.9854 - val_loss: 0.0333 - val_acc: 0.9927
Epoch 6/10
500/500 [==============================] - 67s 133ms/step - loss: 0.0452 - acc: 0.9887 - val_loss: 0.0311 - val_acc: 0.9933
Epoch 7/10
500/500 [==============================] - 72s 144ms/step - loss: 0.0386 - acc: 0.9894 - val_loss: 0.0287 - val_acc: 0.9920
Epoch 8/10
500/500 [==============================] - 6

# Testing

In [125]:
model = keras.models.load_model('final_model_09_0.994.h5')

In [126]:
model.evaluate(test_ds)

63/63 [==============================] - 39s 621ms/step - loss: 0.0253 - acc: 0.9925


[0.025294099003076553, 0.9925000071525574]

In [238]:
path = './images/positive/00825.jpg'
img = load_img(path, target_size=(150, 150))

x = np.array(img)
X = np.array([x])
X.shape

pred = model.predict(X)

1/1 [==============================] - 0s 31ms/step


In [239]:
print('probabilty of positve {}'.format(pred[0]))

probabilty of positve [1.]


In [240]:
pred

array([[1.]], dtype=float32)